In [29]:
import os
import pickle
import torch
import numpy as np
# from transformers import *

MKIAFDTVNVYYLPQFIPICDELAKRGHEVKLVCYSNKNKAQAFEQVLLSFGYEFCWVDDDKAARDLYLKEEPDWIFFGNGFSYLDDIHKVSKTAQLGHGIGPKPSYYHKSSTPMTVRFIEGKMRLAKIRELYPNDEFVQVGFSKLDPLFNNTEPGLKYDELGLDKGKPTLLFAPTFNPSSLECFPDDWPSHFTDFNILIKPHTFTYSREAYKNQRKKLKKWAQFNNTYVATETDISLLPFMKDADILISEASSTLFEFVALSKPVIVCNFFKLKWSYRGIFNYRFEKRFGKDNVIYKNIGLHINDYTELRAAIDKQLNNEHLYKEERKNYTQDHVGPTDGKSSLRIVDYIENN


In [13]:
import logging
logging.basicConfig(level=logging.INFO)

In [14]:
from transformers import BertConfig, BertModel

# custom parameters for BERT model
vocab_size = 24 # Vocabulary size of inputs_ids in BertModel. default=30522
hidden_size = 768 # Size of the encoder layers and the pooler layer, default=768
num_hidden_layers = 12 # Number of hidden layers in the Transformer encoder. default=12
num_attention_heads = 12 # Number of attention heads for each attention layer in the Transformer encoder, default=12
intermediate_size = 3072 # The size of the “intermediate” (i.e., feed-forward) layer in the Transformer encoder. default=3072
hidden_act = "gelu" # The non-linear activation function (function or string) in the encoder and pooler. If string, “gelu”, “relu”, “swish” and “gelu_new” are supported. default="gelu"
hidden_dropout_prob = 0.1 # The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler. default=0.1
attention_probs_dropout_prob = 0.1 # The dropout ratio for the attention probabilities. default=0.1
max_position_embeddings = 512 # The maximum sequence length that this model might ever be used with. Typically set this to something large just in case (e.g., 512 or 1024 or 2048). default=512
type_vocab_size = 2 # The vocabulary size of the token_type_ids passed into BertModel. default=2
initializer_range = 0.02 # The sttdev of the truncated_normal_initializer for initializing all weight matrices. default=0.02
layer_norm_eps = 1e-12 # The epsilon used by LayerNorm. default=1e-12


config = BertConfig(vocab_size_or_config_json_file=vocab_size,
                    hidden_size=hidden_size,
                    num_hidden_layers=num_hidden_layers,
                    num_attention_heads=num_attention_heads,
                    intermediate_size=intermediate_size,
                    hidden_act=hidden_act,
                    hidden_dropout_prob=hidden_dropout_prob,
                    attention_probs_dropout_prob=attention_probs_dropout_prob,
                    max_position_embeddings=max_position_embeddings,
                    type_vocab_size=type_vocab_size,
                    initializer_range=initializer_range,
                    layer_norm_eps=layer_norm_eps)

model = BertModel(config)
# print(model)

model.to('cuda')


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(24, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
 

In [18]:
# load data
data_dir = '/home-3/msomme16@jhu.edu/scratch/shimmer/data/'
CDS_path = os.path.join(data_dir, "CDS_3600.pkl")

with open(CDS_path, 'rb') as f:
    CDS = pickle.load(f)
print(CDS[10])

MKIAFDTVNVYYLPQFIPICDELAKRGHEVKLVCYSNKNKAQAFEQVLLSFGYEFCWVDDDKAARDLYLKEEPDWIFFGNGFSYLDDIHKVSKTAQLGHGIGPKPSYYHKSSTPMTVRFIEGKMRLAKIRELYPNDEFVQVGFSKLDPLFNNTEPGLKYDELGLDKGKPTLLFAPTFNPSSLECFPDDWPSHFTDFNILIKPHTFTYSREAYKNQRKKLKKWAQFNNTYVATETDISLLPFMKDADILISEASSTLFEFVALSKPVIVCNFFKLKWSYRGIFNYRFEKRFGKDNVIYKNIGLHINDYTELRAAIDKQLNNEHLYKEERKNYTQDHVGPTDGKSSLRIVDYIENN


In [52]:
# encode data as GPU tensors
max_aa_seq_length = 5000

def tokenize_aa_seq_murphy10(aa_seq):
    table = {"L":0,
             "V":0,
             "I":0,
             "M":0,
             "C":1,
             "A":2,
             "G":3,
             "S":4,
             "T":4,
             "P":5,
             "F":6,
             "Y":6,
             "W":6,
             "E":7,
             "D":7,
             "N":7,
             "Q":7,
             "K":8,
             "R":8,
             "H":9}
    tokenized = [table[aa] for aa in aa_seq]
    return tokenized
    
example_small = CDS[0:3]

tokens = [tokenize_aa_seq_murphy10(seq) for seq in example_small]
tokens_tensor = torch.zeros(len(tokens), max_aa_seq_length, dtype=torch.uint8)
for i in range(len(tokens)):
    for j in range(len(tokens[i])):
        tokens_tensor[i][j] += tokens[i][j]

tokens_tensor = tokens_tensor.to('cuda')

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [15]:
# example tokenization
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

masked_index = 8
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)
print(tokens_tensor.shape)

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpy7arh3fc
100%|██████████| 231508/231508 [00:00<00:00, 6775814.90B/s]
INFO:transformers.file_utils:copying /tmp/tmpy7arh3fc to cache at /home-3/msomme16@jhu.edu/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /home-3/msomme16@jhu.edu/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:removing temp file /tmp/tmpy7arh3fc
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home-3/msomme16@jhu.edu/.cache/torch/transformers/26bc1

['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']
['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']
tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]])
torch.Size([1, 14])


In [17]:
print(tokens_tensor)

tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]], device='cuda:0')


In [16]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
#     outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpbymbbz2g
100%|██████████| 313/313 [00:00<00:00, 393530.32B/s]
INFO:transformers.file_utils:copying /tmp/tmpbymbbz2g to cache at /home-3/msomme16@jhu.edu/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.file_utils:creating metadata file for /home-3/msomme16@jhu.edu/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.file_utils:removing temp file /tmp/tmpbymbbz2g
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home-3/msomme16@jhu.edu/.cache/torch/transf

henson


In [2]:
# GPT-2

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2-large')


# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()
model.to('cuda')

# original_text = "Martin Steinegger is in Peru because he "
# original_text = "The real reason Steven keeps recruiting German postdocs is "
original_text = "The secret to giving a fun and compelling Joint Lab Meeting presentation is "
# original_text = "Johns Hopkins University is"
# original_text = "UC Berkeley is"
# original_text = "Why did Donald Trump "
# original_text = "Finding genes is easy... The secret is "
# original_text = "The reason I want to get a PhD in Biomedical Engineering is "
# original_text = "Computational gene finding is easy, the real secret is "
# original_text = "Computational gene finding is easy, the problem is "
# original_text = "I came to Johns Hopkins University because "
# original_text = "My experiment does not work because "





text = original_text
for i in range(100): # not the best way to iterate, but it works
#     if text[-1] != ".":
        # Encode a text inputs
        indexed_tokens = tokenizer.encode(text)

        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])

        # If you have a GPU, put everything on cuda
        tokens_tensor = tokens_tensor.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            outputs = model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word (in our case, the word 'man')
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

        text = predicted_text

ImportError: libimf.so: cannot open shared object file: No such file or directory

In [155]:
print("Original text:\t\t", original_text)
print("Completed sentence:\t", predicted_text)

Original text:		 My experiment does not work because 
Completed sentence:	 My experiment does not work because it is not a real experiment.
